# Lembrar de colocar nomes no cabeçalho

### Importando libs e definindo sigmoide

In [240]:
import numpy as np
import matplotlib.pyplot as plt

#sigmoid function
def sigmoid(X):
   return 1/(1+np.exp(-X))

### Função de treinamento da rede

In [241]:
def mlp_n_1_training(x, d, eta, Nt, Ne, Nn, W0N_1, W01_2):
    """
    J_MSE, W_1, W1_2 = mlp_n_1_training(x, d, Nn, eta, Nt, Nb, Ne, W0)
    Saídas:
    J_MSE: valor da função custo ao longo das épocas
    W0_1: vetor de pesos da camada 1. - Cada neurônio tem um peso somente (uma entrada) 
    W1_2: vetor de pesos do neurônio 1 da camada de saída
    Entradas:
    x: sinal de entrada: (apenas uma feature)
    d: sinal desejado
    eta: passo de adaptação
    Nt: número de dados de treinamento
    Ne: número de épocas
    """
    WN_1  = W0N_1.reshape(2,Nn).copy()
    W1_2 = W01_2.reshape(Nn+1,1).copy()

    # inicialização do vetor que contém o valor da função custo
    J_MSE = np.zeros((Ne, 1))

    # Juntamos o vetor de entrada com o sinal desejado e inserimos
    # uma coluna de uns para levar em conta o bias
    Xd = np.hstack((np.ones((Nt, 1)), x.reshape(-1,1), d.reshape(-1,1)))

    v1 = np.zeros((Nt,Nn))
    y1 = np.zeros((Nt,Nn))

    #For das épocas
    for k in range(Ne):

        np.random.shuffle(Xd)
        X = Xd[:, 0:2]
        d = Xd[:, [2]]

        #Cálculo progressivo da camada 1
        for neuron in range(Nn):
            v1[:, neuron] = X@WN_1[:,neuron]
            y1[:, neuron] = sigmoid(v1[:, neuron])

        dphiN_1 = y1*(1 - y1) #dphis da camada 1

        #Neurônio da camada de saída
        X2 = np.hstack((np.ones((Nt, 1)), y1))
        v1_2 = X2@W1_2
        y1_2 = sigmoid(v1_2)
        dphi1_2 = y1_2 * (1 - y1_2)

        #Erro da última camada
        e1_2 = d - y1_2

        #Algoritmo de backpropagation
        delta1_2 = dphi1_2*e1_2
        delta1 = np.zeros((Nt,Nn))

        for neuron in range(Nn):
            delta1[:,neuron] = dphiN_1[:, neuron]*(delta1_2[:,0]*W1_2[neuron+1,0]) #Vetor de deltas dos neurônios da camada 1
            
        #Atualização dos pesos da última camada
        W1_2 = W1_2 + eta*(delta1_2.T @ X2).T / Nt

        #Atualização dos pesos da primeira camada
        WN_1 = WN_1 + eta*(X.T @ delta1) / Nt
        
        J_MSE[k] = (np.linalg.norm(e1_2)) ** 2
    
    return J_MSE, WN_1, W1_2

### Função de teste da rede

In [242]:
def mlp_n_1_testing(x, d, WN_1, W1_2, Nn, Nteste):
   """
   J_MSE,y = redeMLP_teste_21(x, d, W1_1, W2_1, W1_2, Nn, Nteste)
   Saídas:
   J_MSE: valor da função custo no teste
   y: saída da rede MLP
   Entradas:
   x: sinal de entrada
   d: sinal desejado
   Nn: num de neurônios na camada oculta
   WN_1 vetor de pesos da camada 1. - Cada neurônio tem um peso somente (uma entrada) 
   W1_2 vetor de pesos do neurônio 1 da camada de saída
   Nteste: número de dados de teste
   """

   # Adicionando bias
   X_test = np.hstack((np.ones((Nteste, 1)), x.reshape(-1, 1)))
  
   # Output da primeira camada
   v1_test = X_test @ WN_1
   y1_test = sigmoid(v1_test)
   
   # Output da camada de saída
   X2_test = np.hstack((np.ones((Nteste, 1)), y1_test))
   v1_2_test = X2_test @ W1_2
   y1_2_test = sigmoid(v1_2_test)
  
   # Calculando erro quadrático médio
   mse = np.mean((y1_2_test - d) ** 2)
   abse = np.sum(np.abs(y1_2_test - d))
   
   return y1_2_test, mse, abse

# Testando o modelo com as funções propostas

### Criando funções auxiliares:

In [243]:
from typing import Tuple

def get_training_test_set(function: str, num_points: int) -> Tuple:
    if function == 'inverse':
        x = np.random.uniform(1, 100, num_points)
        y = (1/x)

    if function == 'log':
        x = np.random.uniform(1, 10, num_points)
        y = np.log10(x)

    if function == 'exp':
        x = np.random.uniform(1, 10, num_points)
        y = np.exp(-x)
    
    if function == 'sin':
        x = np.random.uniform(0, np.pi/2, num_points)
        y = np.sin(x)
    return x,y

In [244]:
# Função auxiliar para inicializar pesos com inicialização Xavier

def xavier_init(fan_in, fan_out):
   limit = np.sqrt(6 / (fan_in + fan_out))
   return np.random.uniform(-limit, limit, (fan_in, fan_out))

In [245]:
# Função para treinar e testar o modelo

def train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function):
   
    if init == 'xavier':
        # Definindo pesos da camada oculta
        fan_in_1 = 2  # número de features (incluindo bias)
        fan_out_1 = Nn_hid_layer
        W0N_1 = xavier_init(fan_in_1, fan_out_1)

        # Definindo pesos da camda de sáida
        fan_in_2 = Nn_hid_layer + 1 
        fan_out_2 = 1  # Número de neurônios de saída
        W01_2 = xavier_init(fan_in_2, fan_out_2)
    
    else: #Inicialização uniforme
        W0N_1 = 0.2 * np.random.rand(2, Nn_hid_layer) - 0.01
        W01_2 = 0.2 * np.random.rand(Nn_hid_layer+1, 1) - 0.01
    
    #Gerando datasets
    x_train, d_train = get_training_test_set(function, num_pts_train)
    x_test, d_test = get_training_test_set(function, num_pts_test)

    #Trainando o modelo
    J_MSE_train, WN_1, W1_2 = mlp_n_1_training(x=x_train, d=d_train, eta=eta, Nt=num_pts_train, Ne=num_epochs, Nn=Nn_hid_layer, W0N_1=W0N_1, W01_2=W01_2)

    #Testando o modelo
    y_test, mse, abse =  mlp_n_1_testing(x=x_test, d=d_test, WN_1=WN_1, W1_2=W1_2, Nn=Nn_hid_layer, Nteste=num_pts_test)

    return J_MSE_train, x_test, d_test, y_test, mse, abse

In [246]:
#Função para construir gráfico do JMSE de treinamento e os gráficos das funções obtidas com o modelo

def print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse):  
    fig, (ax1, ax2) = plt.subplots(2, 1)

    # Construindo gráficos
    ax1.plot(range(num_epochs), J_MSE_train, label='J_MSE_train')
    title = "JMSE | " + "Ne: " + str(num_epochs) + " Nn: "  + str(Nn_hid_layer) + " eta: " + str(eta)
    ax1.set_title(title)
    ax1.set_ylabel('JMSE_train')
    ax1.legend()
    
    # Gráfico comparativo da resposta dos dados de teste e valores retornados pelo modelo
    ax2.plot(x_test, d_test, label='d (função esperada)', marker='.', linestyle='', alpha=0.5) 
    ax2.plot(x_test, y_test, label='y_test (função obtida)', marker='.', linestyle='', alpha=0.5)
    ax2.set_title(f'Esperado vs resultado do teste\n(mse = {round(mse,5)}, abse = {round(abse, 5)})')
    ax2.legend()
    ax2.set_ylabel(function)
    fig.tight_layout()
    plt.savefig(f'./results/{function}/Ne{num_epochs}_Nn{Nn_hid_layer}_eta{eta}.png', dpi=300)
    # plt.show()

    plt.close()


### Definindo parâmetros constantes:

In [247]:
#Definindo tamanho dos sets de treinamento e teste
num_pts_train = 10000
num_pts_test = 1000

### Exemplo de uso da função construída: (para f = 1/x)

In [248]:
#Definindo hyperparâmetros
num_epochs= 100
Nn_hid_layer= 3
eta= 0.5
function = 'inverse'
init = 'xavier'

#Chamando funcção train_test e imprimindo resultados
J_MSE_train, x_test, d_test, y_test, mse, abse = train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function)
print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse)

### Exemplo de uso da função construída (2) (para f = log10(x))

In [249]:
#Exemplo com log
#Definindo hyperparâmetros
num_epochs= 100
Nn_hid_layer= 3
eta= 0.5
function = 'log'
init = 'xavier'

#Chamando funcção train_test e imprimindo resultados
J_MSE_train, x_test, d_test, y_test, mse, abse = train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function)
print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse)

## Fazendo grid search para encontrar os melhores parâmetros

### Definindo dicionário de grid-search dos hyperparâmetros

In [256]:
grid_search = {
    'num_epochs': [1000, 5000],
    'Nn_hid_layer': [15, 20, 50, 100],
    'eta': [0.6, 1.2],
}
# grid_search = {
#     'num_epochs': [1, 10],
#     'Nn_hid_layer': [3, 4],
#     'eta': [0.1,],
# }

### Fazendo grid search para inversa

In [257]:
import itertools

# Extract the parameter values from the grid_search dictionary
param_values = list(grid_search.values())
function = 'inverse'
init = 'xavier'
mse_min = 100
abse_min = 10**10
# Iterate through all possible combinations of the parameters

for num_epochs, Nn_hid_layer, eta in itertools.product(*param_values):
    J_MSE_train, x_test, d_test, y_test, mse, abse = train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function)
    print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse)
    if abse < abse_min:
        abse_min = abse
        best_params_abse = [num_epochs, Nn_hid_layer, eta]
    if mse < mse_min:
        mse_min = mse
        best_params_mse = [num_epochs, Nn_hid_layer, eta]
print(f'mse_min: {mse_min} \nmelhores hyperparâmetros: {best_params_mse}')
print(f'mse_min: {abse_min} \nmelhores hyperparâmetros: {best_params_abse}')

mse_min: 0.010570667553193172 
melhores hyperparâmetros: [1000, 20, 0.6]
mse_min: 45940.969896287956 
melhores hyperparâmetros: [5000, 100, 0.6]


In [258]:
function = 'log'

# Iterate through all possible combinations of the parameters
for num_epochs, Nn_hid_layer, eta in itertools.product(*param_values):
    J_MSE_train, x_test, d_test, y_test, mse, abse = train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function)
    print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse)
    if abse < abse_min:
        abse_min = abse
        best_params_abse = [num_epochs, Nn_hid_layer, eta]
    if mse < mse_min:
        mse_min = mse
        best_params_mse = [num_epochs, Nn_hid_layer, eta]
print(f'mse_min: {mse_min} \nmelhores hyperparâmetros: {best_params_mse}')
print(f'mse_min: {abse_min} \nmelhores hyperparâmetros: {best_params_abse}')

mse_min: 0.010570667553193172 
melhores hyperparâmetros: [1000, 20, 0.6]
mse_min: 45940.969896287956 
melhores hyperparâmetros: [5000, 100, 0.6]


In [259]:
function = 'exp'
for num_epochs, Nn_hid_layer, eta in itertools.product(*param_values):
    J_MSE_train, x_test, d_test, y_test, mse, abse = train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function)
    print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse)
    if abse < abse_min:
        abse_min = abse
        best_params_abse = [num_epochs, Nn_hid_layer, eta]
    if mse < mse_min:
        mse_min = mse
        best_params_mse = [num_epochs, Nn_hid_layer, eta]
print(f'mse_min: {mse_min} \nmelhores hyperparâmetros: {best_params_mse}')
print(f'mse_min: {abse_min} \nmelhores hyperparâmetros: {best_params_abse}')

mse_min: 0.007566691115223755 
melhores hyperparâmetros: [1000, 15, 1.2]
mse_min: 45940.969896287956 
melhores hyperparâmetros: [5000, 100, 0.6]


In [260]:
function = 'sin'
for num_epochs, Nn_hid_layer, eta in itertools.product(*param_values):
    J_MSE_train, x_test, d_test, y_test, mse, abse = train_test_model(num_pts_train, num_pts_test, num_epochs, Nn_hid_layer, eta, init, function)
    print_results(J_MSE_train, x_test, d_test, y_test, num_epochs, Nn_hid_layer, eta, function, mse, abse)
    if abse < abse_min:
        abse_min = abse
        best_params_abse = [num_epochs, Nn_hid_layer, eta]
    if mse < mse_min:
        mse_min = mse
        best_params_mse = [num_epochs, Nn_hid_layer, eta]
print(f'mse_min: {mse_min} \nmelhores hyperparâmetros: {best_params_mse}')
print(f'mse_min: {abse_min} \nmelhores hyperparâmetros: {best_params_abse}')

mse_min: 0.007566691115223755 
melhores hyperparâmetros: [1000, 15, 1.2]
mse_min: 45940.969896287956 
melhores hyperparâmetros: [5000, 100, 0.6]
